<a href="https://colab.research.google.com/github/orel33/bloc3/blob/master/reseaux/colab/socket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Les Sockets en Python

Ou comment programmer des applications réseaux TCP/IP, avec l'exemple du web (protocole HTTP).

## Avant Propos

* Socket : https://docs.python.org/3/library/socket.html
* HTTP : https://fr.wikipedia.org/wiki/Hypertext_Transfer_Protocol
* Tips : http://aurelien.esnard.emi.u-bordeaux.fr/teaching/doku.php?id=rx3:index
* Cours Moodle DIU : https://moodle1.u-bordeaux.fr/course/view.php?id=4713

## Comprendre le protocole du Web (HTTP)

Avec Wireshark, vérifez que vous comprenez tout de la capture ci-dessous :

=> https://github.com/diu-uf-bordeaux/bloc3/raw/master/reseaux/trace/http.pcap

Plus d'info sur le cours Moodle :  https://moodle1.u-bordeaux.fr/course/view.php?id=4713


## Prérequis

In [9]:
! apt-get -q -y install netcat-openbsd

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  netcat-openbsd
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 39.8 kB of archives.
After this operation, 96.3 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 netcat-openbsd amd64 1.187-1ubuntu0.1 [39.8 kB]
Fetched 39.8 kB in 0s (125 kB/s)
Selecting previously unselected package netcat-openbsd.
(Reading database ... 132723 files and directories currently installed.)
Preparing to unpack .../netcat-openbsd_1.187-1ubuntu0.1_amd64.deb ...
Unpacking netcat-openbsd (1.187-1ubuntu0.1) ...
Setting up netcat-openbsd (1.187-1ubuntu0.1) ...
update-alternatives: using /bin/nc.openbsd to provide /bin/nc (nc) in auto mode
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


## Une requête web à la main...

=> https://fr.wikipedia.org/wiki/Hypertext_Transfer_Protocol

In [10]:
! echo -e "GET / HTTP/1.1\r\nHost: www.perdu.com\r\n\r\n" | netcat -N www.perdu.com 80

HTTP/1.1 200 OK
Date: Wed, 13 Nov 2019 08:41:44 GMT
Server: Apache
Upgrade: h2
Connection: Upgrade
Last-Modified: Thu, 02 Jun 2016 06:01:08 GMT
ETag: "cc-5344555136fe9"
Accept-Ranges: bytes
Content-Length: 204
Vary: Accept-Encoding
Content-Type: text/html

<html><head><title>Vous Etes Perdu ?</title></head><body><h1>Perdu sur l'Internet ?</h1><h2>Pas de panique, on va vous aider</h2><strong><pre>    * <----- vous &ecirc;tes ici</pre></strong></body></html>


Nota Bene : le protocole HTTP nécessite de terminer chaque ligne avec les caractères "\r\n" (CR+LF).

## Chaînes de Caractères

Les fonctions send()/recv() du module *socket* ne manipulent pas des *string* classiques, mais des *byte arrays* !

In [0]:
s0 = "coucou"           # string object       
b0 = b"coucou"          # byte array (notez le prefixe b)
b1 = s0.encode()        # convert string to byte array
s1 = b0.decode()        # convert byte array to string (utf-8 encoding)
print("s0 =", s0)
print("b0 =", b0)

s0 = coucou
b0 = b'coucou'


Les *string* en Python, c'est facile...

=> https://docs.python.org/fr/3/tutorial/inputoutput.html

In [1]:
"Hello, my name is {}!".format("orel")

'Hello, my name is orel!'

## Le Module socket en Python3

=> https://docs.python.org/3.7/library/socket

In [0]:
import socket

Pas facile de trouver des sites webs qui font encore du HTTP 1.0... Notez le message d'erreur !

## Client HTTP 1.1

Dans sa version HTTP 1.1, il est necessaire de préciser le champs **Host** dans l'entête de la requête HTTP. De plus, il est utile de mettre le champs **Connection** à la valeur **close** afin de forcer a fermeture de la connexion TCP/IP, à la fin de la réponse. Voici donc comment préparer une requête HTTP vers le serveur web www.perdu.com (port 80) :

In [12]:
HOST = b'www.perdu.com'
PORT = 80
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))
request = b'GET / HTTP/1.1\r\n'
request += b'Host: ' + HOST + b'\r\n'
request += b'Connection: close\r\n'
request += b'\r\n'
print(request.decode())


GET / HTTP/1.1
Host: www.perdu.com
Connection: close




Envoyer la requête et recevoir les 1024 premiers caractères de la réponse...

In [13]:
s.sendall(request)
answer = s.recv(1024)
print(answer.decode())
s.close()


HTTP/1.1 200 OK
Date: Wed, 13 Nov 2019 08:47:12 GMT
Server: Apache
Upgrade: h2
Connection: Upgrade, close
Last-Modified: Thu, 02 Jun 2016 06:01:08 GMT
ETag: "cc-5344555136fe9"
Accept-Ranges: bytes
Content-Length: 204
Vary: Accept-Encoding
Content-Type: text/html

<html><head><title>Vous Etes Perdu ?</title></head><body><h1>Perdu sur l'Internet ?</h1><h2>Pas de panique, on va vous aider</h2><strong><pre>    * <----- vous &ecirc;tes ici</pre></strong></body></html>



Si l'on souhaite recevoir une réponse de taille variable, il faut "boucler" jusqu'à recevoir une réponse vide (caractéristique de la fermeture de connexion du serveur).

In [0]:
s.sendall(request)

while True:
    answer = s.recv(1024)
    if answer == b'': break
    print(answer.decode('utf-8'), end='')

s.close()


HTTP/1.1 200 OK
Date: Tue, 12 Nov 2019 23:47:09 GMT
Server: Apache
Upgrade: h2
Connection: Upgrade, close
Last-Modified: Thu, 02 Jun 2016 06:01:08 GMT
ETag: "cc-5344555136fe9"
Accept-Ranges: bytes
Content-Length: 204
Vary: Accept-Encoding
Content-Type: text/html

<html><head><title>Vous Etes Perdu ?</title></head><body><h1>Perdu sur l'Internet ?</h1><h2>Pas de panique, on va vous aider</h2><strong><pre>    * <----- vous &ecirc;tes ici</pre></strong></body></html>
